In [1]:
import pandas as pd
import os

env = os.environ.get('Environment')
print(f"Environment: {env!r}")

Environment: 'prod'


In [2]:
%reload_ext jupyter_dmdg

### Step 1: Query all distinct combinations of event_name and location_hierarchy with the most recent event_start

In [3]:
%%athena_to_df --out distinct_last_event_names
SELECT 
    event_name, 
    location_hierarchy, 
    MAX(event_start) AS last_start
FROM "prod_plant_connectivity_prepared_data"."process_events"
GROUP BY 1,2

### Step 2: Only keep the event_name with the most recent event_start

In [4]:
distinct_last_event_names_ordered = distinct_last_event_names.sort_values('last_start', ascending=False)
unique_event_name = distinct_last_event_names_ordered.drop_duplicates(subset='event_name', keep='first')

### Step 3: Select the equipment associated to event_name with the most recent event_start that have secondary_function_id

In [8]:
non_compliant_equipment_due2format_list = unique_event_name['location_hierarchy'][unique_event_name['location_hierarchy'].str.count('-') > 2].tolist()
non_compliant_equipment_due2format = pd.DataFrame({'location_hierarchy': non_compliant_equipment_due2format_list})
non_compliant_equipment_due2format = non_compliant_equipment_due2format.drop_duplicates(subset='location_hierarchy')

### Step 4: Create a table in the dedicated workspace that contains the equipment to be corrected in the S_Egnine equipment catalog because of their non_compliant location hierarchy format

In [9]:
from src.utils import write_into_table
workspace_prefix = 'public-plant-co-data-quality-20230520193210554'

In [ ]:
table_name = 'non_compliant_equipment_due2format'
write_into_table(
    non_compliant_equipment_due2format,
    table_name.lower(),
    workspace_prefix
)